In [1]:
import os 
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import re 
import random
from sklearn.utils import shuffle

In [ ]:
#os.getcwd()

# 8SIDOR corpus parsing 

In [6]:
#zipfile = bz2.BZ2File(filepath) 
#data = zipfile.read() 
filepath = './corpus/attasidor.xml.bz2'
newfilepath = filepath[:-4] 

In [8]:
# max_count parameter - the number of sentences you want to get from the corpus
def parse_xml(filepath, sampling_rate=0.5, max_count = 100000):
    import xml.etree.ElementTree as ET
    import random

    count = 0
    sentences = {}
    
    #newfilepath = './attasidor.xml'
    doc = ET.parse(filepath)
    root = doc.getroot()
    
    for sent in root.iter("sentence"):
        sampling = random.random()
        if sampling > sampling_rate and count < max_count:
            sent_id = sent.attrib.get('id')
            try:
                k = [(x.text, x.get('pos'), x.get('msd'), x.get('lemma'))  \
                     if '\n' not in x.text else (x.get('name'), 'NE', x.get('type')) \
                     for x in sent.findall('./')]
                sentences[sent_id] = k
                count += 1
            
            except: # To catch AttributeError - nonetype data 
                continue
             
    return sentences

In [9]:
sentences = parse_xml(newfilepath)

In [10]:
sentences

{'1c00f0e': [('Och', 'KN', 'KN', '|och|'),
  ('Sverige', 'NE', 'LOC'),
  ('är', 'VB', 'VB.PRS.AKT', '|vara|'),
  ('det', 'DT', 'DT.NEU.SIN.DEF', '|den|en|'),
  ('allra', 'AB', 'AB', '|allra|'),
  ('mest', 'AB', 'AB.SUV', '|mycket|'),
  ('barnvänliga', 'JJ', 'JJ.POS.UTR+NEU.SIN.DEF.NOM', '|barnvänlig|'),
  ('landet', 'NN', 'NN.NEU.SIN.DEF.NOM', '|land|'),
  ('för', 'PP', 'PP', '|för|'),
  ('människor', 'NN', 'NN.UTR.PLU.IND.NOM', '|människa|'),
  ('som', 'HP', 'HP.-.-.-', '|'),
  ('har', 'VB', 'VB.PRS.AKT', '|ha|'),
  ('barn', 'NN', 'NN.NEU.PLU.IND.NOM', '|barn|'),
  ('.', 'MAD', 'MAD', '|')],
 '1177043': [('Centerpartiets', 'NE', 'ORG'),
  ('ledare', 'NN', 'NN.UTR.SIN.IND.NOM', '|ledare|'),
  ('Annie Lööf', 'NE', 'PRS'),
  ('tycker', 'VB', 'VB.PRS.AKT', '|tycka|'),
  ('att', 'SN', 'SN', '|att|'),
  ('Moderaterna', 'NE', 'ORG'),
  (',', 'MID', 'MID', '|'),
  ('Kristdemokraterna och Liberalerna', 'NE', 'PRS'),
  ('inte', 'AB', 'AB', '|inte|'),
  ('håller', 'VB', 'VB.PRS.AKT', '|hålla|'),

# Pre-defined rules for synthetic data augmentation

In [11]:
import random

def insertion(token): # character-level insertion 
    
    word = token[0] # token[0] : word
    random_num = random.randint(0, len(word)-1)
    word_list = list(word)
    word_list.insert(random_num, word_list[random_num])
    
    return ''.join(word_list)
    

def deletion(token):
    word = token[0]
    random_num = random.randint(0, len(word)-1)
    word_list = list(word)
    
    # morphological level 
    if word_list[-2:] =='en':
        word_list[-2:] == ''
    elif word_list[-2:] == 'et':
        word_list[-2:] == ''  
    elif word_list[-2:] == 'na':
        word_list[-2:] == ''
    elif 'nn' in word:
        word = word.replace('nn', 'n')
        word_list = list(word)
    elif 'll' in word:
        word = word.replace('ll', 'l')
        word_list = list(word)
    elif 'mm' in word:
        word = word.replace('mm', 'm')
        word_list = list(word)
    elif 'ss' in word:
        word = word.replace('ss', 's')
        word_list = list(word)
    
    else: # character level deletion     
        word_list.pop(random_num)
    
    return ''.join(word_list)


def substitution(token):
    
    swap_words = ['de', 'De', 'får', 'måste', 'ville', 'ska', 'er', 'ar', 'et', 'en']
    word = token[0].strip()
    random_num = random.randint(0, len(word)-1)
    word_list = list(word)
    
    # WORD LEVEL SUBSTITUTION 
    if word == 'de':
        word = 'dem'
        word_list = list(word)
    elif word == 'De':
        word = 'Dem'
        word_list = list(word)
    elif word == 'får':
        word = 'måste'
        word_list = list(word)
    elif word == 'måste':
        word = 'får'
        word_list = list(word)
    elif word == 'ville':
        word = 'ska'
        word_list = list(word)
    elif word == 'ska':
        word = 'ville'
        word_list = list(word)
    elif word == 'vill':
        word = 'kommer att'
        word_list = list(word)
        
    # MORPHOLOGICAL LEVEL SUBSTITUTION                     
    elif word[-2:] == 'er':
        word[-2:] == 'ar'
        word_list == list(word)
    elif word[-2:] == 'ar':
        word[-2:] == 'er'
        word_list = list(word)
    elif word[-2:] == 'en':
        word[-2:] == 'et'
        word_list = list(word)
    elif word[-2:] == 'et':
        word[-2:] == 'en'
        word_list = list(word)
    
    # CHARACTER LEVEL SUBSTITUTION 
    elif 'å' in word:
        word = word.replace('å', 'o')
        word_list = list(word)
    elif 'o' in word:
        word = word.replace('o', 'å')
        word_list = list(word)
    elif 'e' in word:
        word = word.replace('e', 'i')
        word_list = list(word)
    elif 'i' in word:
        word = word.replace('i', 'e')
        word_list = list(word)
    elif 'c' in word:
        word = word.replace('c', 'k')
        word_list = list(word)
    elif 'k' in word:
        word = word.replace('k', 'c')
        word_list = list(word)
    else:
        if random_num != 0:
            word_list[random_num-1], word_list[random_num] = word_list[random_num], word_list[random_num-1]
        else:
            word_list[random_num], word_list[random_num+1] = word_list[random_num+1], word_list[random_num]
    return "".join(word_list)
    
       
def lemmatization(token):
    lemma_list = token[3].split('|')
    lemma_list = [lemma for lemma in lemma_list if len(lemma) >= 1]
    
    if len(lemma_list) == 1 and len(lemma_list[0]) > 0 :
        token = lemma_list[0]    
    elif len(lemma_list) > 1:
        random_num = random.random()
        if random_num > 0.5:
            token = lemma_list[0]
        else:
            token = lemma_list[1]                
    return token

In [12]:
backtranslated_sent = []
original_sent = []

count = 0
for key, value in sentences.items():
    pre_defined_rules = ['lemmatization', 'substitution', 'deletion', 'insertion', 'pass']  
    
    clean_sent = ''
    error_sent = ''
    
    for token in value:
        pick_rule = random.choices(pre_defined_rules, weights=[0.05, 0.05, 0.05, 0.05, 0.8]) #probability of 0.05 for the rules
        
        try:
            clean_sent = clean_sent + token[0] + ' '
            if token[0].isalpha() and len(token[0]) >= 3 and token[0].islower():
     
                if pick_rule == ['lemmatization']:
                    try:
                        if token[3] != '|':
                            tok = lemmatization(token)
                            error_sent = error_sent + tok + ' '
                        else:
                            error_sent = error_sent + token[0] + ' '

                    except IndexError:
                        error_sent = error_sent + token[0] + ' '

                elif pick_rule == ['substitution']:
                    tok = substitution(token)
                    error_sent = error_sent + tok + ' '

                elif pick_rule == ['deletion']:
                    tok = deletion(token)
                    error_sent = error_sent + tok + ' '

                elif pick_rule == ['insertion']:
                    tok = insertion(token)
                    error_sent = error_sent + tok + ' '

                else: # pass 
                    error_sent = error_sent + token[0] + ' '
                    
            else: # if word is not alphabet 
                error_sent = error_sent + token[0] + ' '
        except TypeError:
            continue

    original_sent.append(clean_sent.strip())
    backtranslated_sent.append(error_sent.strip())
    count += 1

    if count % 10000 == 0:
        print(f'sentence number {count} clear')

sentence number 10000 clear
sentence number 20000 clear
sentence number 30000 clear
sentence number 40000 clear
sentence number 50000 clear
sentence number 60000 clear
sentence number 70000 clear
sentence number 80000 clear
sentence number 90000 clear
sentence number 100000 clear


In [13]:
pseudo_parallel = pd.DataFrame(
    {'src': backtranslated_sent, # psuedo error sentences 
     'tgt': original_sent # clean original sentence 
    })

In [14]:
pseudo_parallel.head()

,src,tgt
0,Och Sverige är det allra mest barnvänliga land...,Och Sverige är det allra mest barnvänliga land...
1,Centerpartiets ledare Annie Lööf tycker att Mo...,Centerpartiets ledare Annie Lööf tycker att Mo...
2,Hittills har nästan 300 hemlösa fått vaccinet .,Hittills har nästan 300 hemlösa fått vaccinet .
3,Jonathan Eriksson gjorde ett av mål .,Jonathan Eriksson gjorde ett av målen .
4,Nu har en åklagare bestämt att undersökningen ...,Nu har en åklagare bestämt att undersökningen ...


In [16]:
pseudo_parallel = pseudo_parallel.query("src != tgt") # deleting rows when source-target sentences are identical

In [17]:
pseudo_parallel

,src,tgt
1,Centerpartiets ledare Annie Lööf tycker att Mo...,Centerpartiets ledare Annie Lööf tycker att Mo...
3,Jonathan Eriksson gjorde ett av mål .,Jonathan Eriksson gjorde ett av målen .
6,Det vara jägarna i Västmanland som misslyckades .,Det var jägarna i Västmanland som misslyckades .
7,Då minskadi antal lärare i skolan och antalet ...,Då minskade antalet lärare i skolan och antale...
8,"I trädgården , i park och ute i naaturen .","I trädgården , i parken och ute i naturen ."
...,...,...
99980,Henrik Dorsin som är med i gruppen fick desuto...,Henrik Dorsin som är med i gruppen fick dessut...
99987,"Den 30 oktober stängde många affärer , restaur...","Den 30 oktober stängde många affärer , restaur..."
99989,"– Det kändes fint att sätta den i mål , sade h...","– Det kändes fint att sätta den i mål , sade h..."
99996,"I dag , måndag , kommer facklan fram till huvu...","I dag , måndag , kommer facklan fram till huvu..."


In [18]:
pseudo_parallel_after = shuffle(pseudo_parallel)
pseudo_parallel_after.reset_index(inplace=True, drop=True)

# pseudo_parallel.to_csv('./data/synthetic_data/all_synthetic_sents.tsv', # To save CSV 
#                        header=None, columns=['src', 'tgt'], 
#                        index=None, 
#                        sep='\t')